# Using OSM Fetched Data to Perform Network Analysis. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# importing necessary modules

## Step 1

### Function to automate query in Open Streeet Map using Overpass Api. 
More information about how to geocode with OSM can be found here. https://wiki.openstreetmap.org/wiki/Overpass_API?form=MG0AV3#Introduction

#### For correct Query Input visit: https://wiki.openstreetmap.org/wiki/Map_features

##### Store names like "Target", "Wallmart" wouldn't work in this case. We have to be more generic or specify Brand. 

In [ ]:
def get_locations(categories, queries, cities, state, country, brand=None):
    """Fetch locations from OpenStreetMap using Overpass API."""
    overpass_url = "http://overpass-api.de/api/interpreter"
    all_locations = []

    # Construct the optional brand filter
    brand_filter = f'["brand"="{brand}"]' if brand else ''

    # Handle multiple categories, queries, and cities
    if not categories or not queries or not cities:
        print("No categories, queries, or cities provided.")
        return []

    for category in categories:
        for city in cities:
            for query in queries:
                overpass_query = f"""
                [out:json];
                area[name="{city}"]->.searchArea;
                (
                  node["{category}"="{query}"]{brand_filter}(area.searchArea);
                  way["{category}"="{query}"]{brand_filter}(area.searchArea);
                  relation["{category}"="{query}"]{brand_filter}(area.searchArea);
                );
                out center;
                """
                
                try:
                    response = requests.get(overpass_url, params={'data': overpass_query})
                    response.raise_for_status()
                    data = response.json()
                    all_locations.extend(data.get("elements", []))
                except requests.exceptions.RequestException as e:
                    print(f"Request error for {category}={query} in {city}: {e}")
                except requests.exceptions.JSONDecodeError:
                    print(f"Error decoding JSON response from API for {category}={query} in {city}.")

    return all_locations

In [ ]:
def plot_locations(data, city, state, country):
    """Plot locations on a Folium map."""
    if not data:
        print("No locations found.")
        return None

    # Extract the first valid location for map centering
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            m = folium.Map(location=[lat, lon], zoom_start=12)
            break
    else:
        print("No valid coordinates found.")
        return None

    # Add markers
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            folium.Marker([lat, lon], popup=f"{name} ({lat}, {lon})").add_to(m)

    return m

In [ ]:
def display_locations(data):
    """Display location names with coordinates in a DataFrame."""
    locations = []
    for place in data:
        lat = place.get('lat') or (place.get('center', {}).get('lat'))
        lon = place.get('lon') or (place.get('center', {}).get('lon'))
        if lat and lon:
            name = place.get('tags', {}).get('name', 'Unknown')
            locations.append([name, lat, lon])
    
    df = pd.DataFrame(locations, columns=['Name', 'Latitude', 'Longitude'])
    return df

In [ ]:
# Example Query Parameters
category = .....
queries = ......
cities = ....
state = ....
country = ....
brand = None  # Change to "Walmart" or "Target" if needed or None 

# Fetch Data
data = get_locations(category, queries, cities, state, country, brand)

# Plot Data on Map
map_result = plot_locations(data, cities[0] if cities else None, state, country)

# Display DataFrame of Locations
df_locations = display_locations(data)

# Display the map and data
if map_result:
    display(map_result)

## Step 2

#### Using OSMNX module fetch the data of an area of Interest 

In [ ]:
# Hint: ox.graph_from_place

In [ ]:
# Plot the Road Network of the Area

In [ ]:
# Add the Edge Speeds and Travel Times

In [ ]:
# Extract node positions and calculate the centroid
node_positions = np.array([[data['x'], data['y']] for _, data in graph.nodes(data=True)])
centroid = node_positions.mean(axis=0)
centroid_node = ox.nearest_nodes(graph, X=centroid[0], Y=centroid[1])

In [ ]:
# Define a colormap to assign colors dynamically
# get the total number of locations and assign it to a variable
#colormap = cm.get_cmap('tab20', ????)
colors = [colormap(i) for i in range(num_locations)]

In [ ]:
# Create lists to store travel distances and times
travel_distances = []
travel_times = []

# Plot the graph
fig, ax = ox.plot_graph(graph, bgcolor="white", node_size=0, edge_color="black", edge_linewidth=0.5, show=False, close=False)

# Loop through each location and plot the shortest path to the centroid
for idx, row in df_locations.iterrows():
    
    # Find the nearest node to the location coordinates
    
    # Calculate the shortest path route
    
    # Calculate the travel distance and time

    # Append the results to the lists
    travel_distances.append(travel_distance)
    travel_times.append(travel_time)
    
    # Plot the route with different colors
    ox.plot_graph_route(graph, route, route_color=colors[idx], route_linewidth=5, ax=ax, show=False, close=False)

# Highlight the centroid with a red dot
ax.scatter(centroid[0], centroid[1], ?????)

# Add a legend

In [ ]:
# Create a DataFrame to display the travel distances and times
results_df = pd.DataFrame({
    'Name': df_locations['Name'],
    'Travel Distance (meters)': travel_distances,
    'Travel Time (seconds)': travel_times
})

# Display the results DataFrame
display(results_df)